In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, mutual_info_classif

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, \
    classification_report

import matplotlib.pyplot as plt

In [3]:
labels = pd.read_csv('../data/Label.csv')
data = pd.read_csv('../data/Data.csv')

In [4]:
print("Data shape before processing:", data.shape)

# Check for missing values in each column
missing_values_column = data.isnull().sum()
if missing_values_column.sum() > 0:
    print("Columns with missing values:")
    print(missing_values_column[missing_values_column > 0])  # Display columns with missing values
else:
    print("No missing values found in the data.")

# Find the number of unique values in each column
unique_values = data.nunique()
print("Number of unique values in each column:")
print(unique_values)

# Drop columns with only one unique value
for col in data.columns:
    if data[col].nunique() == 1:
        data.drop(col, axis=1, inplace=True)
        print(f"Dropped column: {col}")

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels['Label'], test_size=0.2, random_state=42)

Data shape before processing: (447915, 76)
No missing values found in the data.
Number of unique values in each column:
Flow Duration                 174373
Total Fwd Packet                 847
Total Bwd packets                928
Total Length of Fwd Packet      7697
Total Length of Bwd Packet      9322
                               ...  
Active Min                      1778
Idle Mean                       2280
Idle Std                         611
Idle Max                        2280
Idle Min                        2280
Length: 76, dtype: int64
Dropped column: Bwd PSH Flags
Dropped column: Fwd URG Flags
Dropped column: Bwd URG Flags
Dropped column: URG Flag Count
Dropped column: CWR Flag Count
Dropped column: ECE Flag Count
Dropped column: Fwd Bytes/Bulk Avg
Dropped column: Fwd Packet/Bulk Avg
Dropped column: Fwd Bulk Rate Avg


using mutual_info_classifier

In [5]:
best_features = SelectKBest(score_func=mutual_info_classif, k=30).fit(data, labels['Label'])

In [6]:
# Get the indices of the selected features
selected_features = best_features.get_support(indices=True)

# Get the scores of the features
feature_scores = best_features.scores_

# Map selected feature indices to their column names
selected_feature_names = data.columns[selected_features]

# Output selected feature names and their scores
print("Selected features and their scores:")
# for feature_name, score in zip(selected_feature_names, feature_scores[selected_features]):
#     print(f"{feature_name}: {score}")

print(selected_feature_names)


Selected features and their scores:
Index(['Flow Duration', 'Total Fwd Packet', 'Total Length of Fwd Packet',
       'Total Length of Bwd Packet', 'Fwd Packet Length Max',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Fwd IAT Mean',
       'Bwd IAT Mean', 'Fwd Header Length', 'Bwd Header Length',
       'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Max',
       'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance',
       'ACK Flag Count', 'Down/Up Ratio', 'Average Packet Size',
       'Fwd Segment Size Avg', 'FWD Init Win Bytes', 'Bwd Init Win Bytes',
       'Fwd Seg Size Min'],
      dtype='object')


In [7]:
X_train_reduced = X_train[selected_feature_names]
X_test_reduced = X_test[selected_feature_names]

# Print the shape of the reduced training and testing datasets
print("Reduced Training set shape:", X_train_reduced.shape)
print("Reduced Testing set shape:", X_test_reduced.shape)

Reduced Training set shape: (358332, 30)
Reduced Testing set shape: (89583, 30)


In [8]:
rf_classifier = RandomForestClassifier(random_state=42)

In [10]:
rf_classifier.fit(X_train_reduced, y_train)

RandomForestClassifier(random_state=42)

In [13]:
y_yeild=rf_classifier.predict(X_test_reduced)

In [15]:
print("\nClassification Report:")
report = classification_report(y_test, y_yeild, target_names=[f"Class {i}" for i in np.unique(y_test)])
print(report)


Classification Report:
              precision    recall  f1-score   support

     Class 0       0.99      0.98      0.99     71701
     Class 1       0.29      0.37      0.32        68
     Class 2       0.89      0.47      0.62       104
     Class 3       0.63      0.31      0.41       916
     Class 4       0.75      0.79      0.77      6259
     Class 5       0.65      0.80      0.72      5849
     Class 6       0.84      0.75      0.79       867
     Class 7       0.76      0.69      0.72      3392
     Class 8       0.50      0.38      0.43       381
     Class 9       0.59      0.28      0.38        46

    accuracy                           0.93     89583
   macro avg       0.69      0.58      0.62     89583
weighted avg       0.94      0.93      0.93     89583

